##Системы искусственного интеллекта

##ЛР5. Ансамбли, случайный лес, градиентный бустинг
[20 + 2 баллов]

Чураков А. А. P3331 409856

--------

Начнём с датасета, который представляет собой данные о подержанных автомобилях, продаваемых в Великобритании. Он включает в себя характеристики автомобилей (пробег, год выпуска, тип двигателя и т. п.) и их цену (целевая переменная)

[Описание датасета на kaggle](https://www.kaggle.com/datasets/muhammadawaistayyab/used-cars-prices-in-uk)

Прежде чем начать, импортируем некоторые модули и загрузим данные

In [ ]:
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

In [ ]:
! gdown 1Cdek-lsS3-XiDFtD5PzgXNsy3gvd_Rd-

Downloading...
From: https://drive.google.com/uc?id=1Cdek-lsS3-XiDFtD5PzgXNsy3gvd_Rd-
To: /content/used_cars_UK.csv
100% 290k/290k [00:00<00:00, 6.58MB/s]


##  <font color='DarkOrange'>**Задача 1 [1 балл]**</font>

Подготовка данных

Считайте датасет и предобработайте его:
1. Удалите ненужные столбцы: `'Unnamed: 0'`, `'Service history'`, `'title'`
2. Заполните пропуски в вещественных переменных. Пустые значения в поле `Previous Owners` заполните `0`, остальные поля заполните `средними значениями`
3. Обработайте `категориальные переменные:` объедините редкие классы в `"Other"`, после этого преобразуйте `категориальные переменные` в вещественные с помощью [pd.get_dummies()](https://pandas.pydata.org/docs/reference/api/pandas.get_dummies.html) или [One-Hot Encoder](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html)
    * Обратите внимание на поле `Engine`. Чем оно отличается от `Fuel type` или `Body type`?
4. Разбейте весь датасет на `train/test` в пропорции `80/20`, зафиксируйте `random_state=42`
    * Для `X_train`, `X_test`, `y_train`, `y_test` обновите индексы через `df.reset_index(drop=True)`

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

df = pd.read_csv("/content/used_cars_UK.csv")

drop_cols = ['Unnamed: 0', 'Service history', 'title']
df = df.drop(columns=drop_cols, errors='ignore')

num_cols = df.select_dtypes(include=['float64', 'int64']).columns.tolist()

if 'Previous Owners' in df.columns:
    df['Previous Owners'] = df['Previous Owners'].fillna(0)

for col in num_cols:
    if col != 'Previous Owners':
        df[col] = df[col].fillna(df[col].mean())

cat_cols = df.select_dtypes(include=['object']).columns.tolist()

def collapse_rare(series, threshold=0.01):
    freq = series.value_counts(normalize=True)
    rare_values = freq[freq < threshold].index
    return series.replace(rare_values, "Other")

for col in cat_cols:
    df[col] = collapse_rare(df[col])

if 'Engine' in df.columns:
    df['Engine'] = df['Engine'].astype(str).str.extract(r'(\d\.\d+)').astype(float)

df = pd.get_dummies(df, columns=cat_cols, drop_first=True)

y = df['Price']
X = df.drop(columns=['Price'])

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

X_train = X_train.reset_index(drop=True)
X_test  = X_test.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
y_test  = y_test.reset_index(drop=True)

X_train.head()


,Mileage(miles),Registration_Year,Previous Owners,Doors,Seats,Fuel type_Other,Fuel type_Petrol,Fuel type_Petrol Hybrid,Body type_Coupe,Body type_Estate,...,Engine_1.8,Engine_2.0,Engine_2.1,Engine_2.2,Engine_3.0,Gearbox_Manual,Emission Class_Euro 4,Emission Class_Euro 5,Emission Class_Euro 6,Emission Class_Other
0,55301,2016,0.0,5.0,5.0,False,False,False,False,False,...,False,False,False,False,False,True,False,False,True,False
1,49482,2017,2.0,5.0,5.0,False,True,False,False,False,...,False,False,False,False,False,True,False,False,True,False
2,122000,2010,5.0,5.0,5.0,False,True,False,False,False,...,False,False,False,False,False,True,False,True,False,False
3,111695,2011,3.0,5.0,5.0,False,True,False,False,False,...,False,False,False,False,False,True,True,False,False,False
4,124000,2009,0.0,3.0,5.0,False,True,False,False,False,...,True,False,False,False,False,True,True,False,False,False


 ## <font color='DarkOrange'>**Задача 2 [3 балла]**</font>

Постройте бэггинг над решающими деревьями

Постройте бэггинг с решающими деревьями для задачи регрессии

Используйте [BaggingRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.BaggingRegressor.html) из sklearn, параметры по умолчанию, зафиксировав `random_state=42`, обучите ансамбль, на тесте посмотрите метрики RMSE и MAPE

In [ ]:
from sklearn.ensemble import BaggingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
import numpy as np

model = BaggingRegressor(
    estimator=DecisionTreeRegressor(),
    random_state=42
)

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mape = mean_absolute_percentage_error(y_test, y_pred) * 100  # в %

print(f"RMSE: {rmse:.3f}")
print(f"MAPE: {mape:.2f}%")

RMSE: 1797.823
MAPE: 23.49%


Подберите оптимальные гиперпараметры с помощью [GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) для бэггинга

"Пробегитесь" по следующим параметрам:
* `n_estimators: [10, 25, 50, 100, 200]`
* `max_features: [0.2, 0.5, 0.7, 1.0]`
* `max_samples: [0.5, 0.75, 1.0]`
* `base_estimator__max_depth: [4, 6, 8, 10, 20, None]` — этот параметр оптимизирует глубину базового алгоритма (дерева)

В качестве метрики оптимизации для GridSearchCV используйте `MSE` (при инициализации `GridSearchCV()` будет `scoring='neg_mean_squared_error'`)

In [ ]:
from sklearn.ensemble import BaggingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV

bagging = BaggingRegressor(
    estimator=DecisionTreeRegressor(),
    random_state=42,
)

param_grid = {
    "n_estimators": [10, 25, 50, 100, 200],
    "max_features": [0.2, 0.5, 0.7, 1.0],
    "max_samples": [0.5, 0.75, 1.0],
    "estimator__max_depth": [4, 6, 8, 10, 20, None]
}

grid = GridSearchCV(
    estimator=bagging,
    param_grid=param_grid,
    scoring='neg_mean_squared_error',
    cv=2,
    n_jobs=-1,
    verbose=1
)

grid.fit(X_train, y_train)

print("Лучшие параметры:")
print(grid.best_params_)

print("\nЛучшее значение MSE (со знаком минус):")
print(grid.best_score_)

Fitting 2 folds for each of 360 candidates, totalling 720 fits
Лучшие параметры:
{'estimator__max_depth': None, 'max_features': 0.7, 'max_samples': 0.75, 'n_estimators': 200}

Лучшее значение MSE (со знаком минус):
-2302545.031966117


Обучите модель с лучшими параметрами и оцените на тесте.
Насколько улучшилось качество модели по сравнению с моделью с параметрами по умолчанию?

In [ ]:
from sklearn.ensemble import BaggingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
import numpy as np

default_model = BaggingRegressor(
    estimator=DecisionTreeRegressor(),
    random_state=42
)

default_model.fit(X_train, y_train)
y_pred_default = default_model.predict(X_test)

rmse_default = np.sqrt(mean_squared_error(y_test, y_pred_default))
mape_default = mean_absolute_percentage_error(y_test, y_pred_default) * 100

print("=== Default model ===")
print(f"RMSE: {rmse_default:.3f}")
print(f"MAPE: {mape_default:.2f}%")


=== Default model ===
RMSE: 1797.823
MAPE: 23.49%


In [ ]:
best_model = grid.best_estimator_

best_model.fit(X_train, y_train)
y_pred_best = best_model.predict(X_test)

rmse_best = np.sqrt(mean_squared_error(y_test, y_pred_best))
mape_best = mean_absolute_percentage_error(y_test, y_pred_best) * 100

print("\n=== Best model ===")
print(f"RMSE: {rmse_best:.3f}")
print(f"MAPE: {mape_best:.2f}%")


=== Best model ===
RMSE: 1811.639
MAPE: 25.67%


In [ ]:
print("\n=== Improvement ===")
print(f"RMSE improvement: {rmse_default - rmse_best:.3f}")
print(f"MAPE improvement: {mape_default - mape_best:.2f}%")


=== Improvement ===
RMSE improvement: -13.817
MAPE improvement: -2.18%


## <font color='DarkOrange'>**Задача 3 [3 баллa]**</font>  

Блендинг (вспомните, какое требуется условие на коэффициенты линейной модели как мета-алгоритма)

Постройте 3 базовые модели
* линейная регрессия,
* kNN (k > 1),
* дерево решений (зафиксируйте random_state=42)

Проверьте модели:

* Обучите модели на всём обучающем датасете.
* Оцените качество моделей `(RMSE / MAPE)` на тесте

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
import numpy as np

lin = LinearRegression()
lin.fit(X_train, y_train)

y_pred_lin = lin.predict(X_test)

rmse_lin = np.sqrt(mean_squared_error(y_test, y_pred_lin))
mape_lin = mean_absolute_percentage_error(y_test, y_pred_lin) * 100

print("=== Linear Regression ===")
print(f"RMSE: {rmse_lin:.3f}")
print(f"MAPE: {mape_lin:.2f}%")

=== Linear Regression ===
RMSE: 2324.829
MAPE: 37.07%


In [ ]:
from sklearn.neighbors import KNeighborsRegressor

knn = KNeighborsRegressor(n_neighbors=5)
knn.fit(X_train, y_train)

y_pred_knn = knn.predict(X_test)

rmse_knn = np.sqrt(mean_squared_error(y_test, y_pred_knn))
mape_knn = mean_absolute_percentage_error(y_test, y_pred_knn) * 100

print("\n=== kNN (k=5) ===")
print(f"RMSE: {rmse_knn:.3f}")
print(f"MAPE: {mape_knn:.2f}%")


=== kNN (k=5) ===
RMSE: 3182.321
MAPE: 51.10%


In [ ]:
from sklearn.tree import DecisionTreeRegressor

tree = DecisionTreeRegressor(random_state=42)
tree.fit(X_train, y_train)

y_pred_tree = tree.predict(X_test)

rmse_tree = np.sqrt(mean_squared_error(y_test, y_pred_tree))
mape_tree = mean_absolute_percentage_error(y_test, y_pred_tree) * 100

print("\n=== Decision Tree ===")
print(f"RMSE: {rmse_tree:.3f}")
print(f"MAPE: {mape_tree:.2f}%")


=== Decision Tree ===
RMSE: 2183.279
MAPE: 31.98%


Разделите обучающий датасет на 2 подвыборки в соотношении 80/20: одна будет использована для обучения базовой модели `(X_base_train, y_base_train)`, другая для <font color='CornflowerBlue'>**мета-модели**</font> `(X_meta_train, y_meta_train)`

In [ ]:
from sklearn.model_selection import train_test_split

X_base_train, X_meta_train, y_base_train, y_meta_train = train_test_split(
    X_train,            # уже существующий train после первого split
    y_train,
    test_size=0.2,      # 20% на мета-модель
    random_state=42
)

X_base_train = X_base_train.reset_index(drop=True)
X_meta_train = X_meta_train.reset_index(drop=True)
y_base_train = y_base_train.reset_index(drop=True)
y_meta_train = y_meta_train.reset_index(drop=True)

X_base_train.shape, X_meta_train.shape

((2358, 31), (590, 31))

Обучите блендинг и оцените его качество
1. Обучите базовые модели на `X_base_train`
2. Сделайте предсказания базовых моделей на `X_meta_trai`n. Объедините все предсказания в один `DataFrame`, замените исходный `X_meta_train` на полученный `df` из прогнозов базовых моделей
3. Обучите мета-модель — `линейную регрессию` на обновлённом `X_meta_train`
4. Сделайте прогнозы базовых моделей на `X_test`. Объедините все прогнозы в один `DataFrame` — `X_meta_test`
5. Сделайте прогноз мета-модели на `X_meta_test`, оцените качество полученной модели с помощью `y_test`
<font color='LightSteelBlue'>**Подсказка.**</font> Воспользуйтесь любым готовым решением для блендинга

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor

# Базовые модели
m1 = LinearRegression()
m2 = KNeighborsRegressor(n_neighbors=5)
m3 = DecisionTreeRegressor(random_state=42)

# Обучение на base-подвыборке
m1.fit(X_base_train, y_base_train)
m2.fit(X_base_train, y_base_train)
m3.fit(X_base_train, y_base_train)
pass

In [ ]:
import pandas as pd

meta_features = pd.DataFrame({
    "linreg": m1.predict(X_meta_train),
    "knn":    m2.predict(X_meta_train),
    "tree":   m3.predict(X_meta_train)
})

y_meta = y_meta_train

In [ ]:
meta_model = LinearRegression()
meta_model.fit(meta_features, y_meta)
pass

In [ ]:
X_meta_test = pd.DataFrame({
    "linreg": m1.predict(X_test),
    "knn":    m2.predict(X_test),
    "tree":   m3.predict(X_test)
})

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
import numpy as np

y_pred_blend = meta_model.predict(X_meta_test)

rmse_blend = np.sqrt(mean_squared_error(y_test, y_pred_blend))
mape_blend = mean_absolute_percentage_error(y_test, y_pred_blend) * 100

print(f"Blending RMSE: {rmse_blend:.3f}")
print(f"Blending MAPE: {mape_blend:.2f}%")

Blending RMSE: 1991.475
Blending MAPE: 30.72%


Оцените качество итоговой модели.
Сравните её с базовыми моделями и с бэггингом над решающими деревьями

In [ ]:
import pandas as pd

results = pd.DataFrame({
    "Model": [
        "Linear Regression",
        "kNN",
        "Decision Tree",
        "Bagging (default)",
        "Bagging (best params)",
        "Blending"
    ],
    "RMSE": [
        rmse_lin,
        rmse_knn,
        rmse_tree,
        rmse_default,
        rmse_best,
        rmse_blend
    ],
    "MAPE": [
        mape_lin,
        mape_knn,
        mape_tree,
        mape_default,
        mape_best,
        mape_blend
    ]
})

print(results)


                   Model         RMSE       MAPE
0      Linear Regression  2324.828819  37.065643
1                    kNN  3182.321332  51.104585
2          Decision Tree  2183.278880  31.975034
3      Bagging (default)  1797.822660  23.487035
4  Bagging (best params)  1811.639498  25.670542
5               Blending  1991.475019  30.724135


## <font color='DarkOrange'>**Задача 4 [3 баллa]**</font> + <font color='MediumSeaGreen'>**[2 бонусных балла]**</font>

Стекинг - усложняем мета-алгоритм

Обучите стекинг
1. Воспользуйтесь готовым классом [StackingRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.StackingRegressor.html) из sklearn
2. В качестве базовых моделей инициализируйте `линейную регрессию, kNN, решающее дерево`
3. В качестве мета-модели (`final_estimator`) используйте `линейную регрессию`
4. Обучите построенную модель на `X_train`, сделайте прогнозы на `X_test`

In [ ]:
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
import numpy as np

estimators = [
    ('linreg', LinearRegression()),
    ('knn', KNeighborsRegressor(n_neighbors=5)),
    ('tree', DecisionTreeRegressor(random_state=42))
]

final_estimator = LinearRegression()

stack_model = StackingRegressor(
    estimators=estimators,
    final_estimator=final_estimator,
    n_jobs=-1
)

stack_model.fit(X_train, y_train)

y_pred_stack = stack_model.predict(X_test)

rmse_stack = np.sqrt(mean_squared_error(y_test, y_pred_stack))
mape_stack = mean_absolute_percentage_error(y_test, y_pred_stack) * 100

print(f"Stacking RMSE: {rmse_stack:.3f}")
print(f"Stacking MAPE: {mape_stack:.2f}%")

Stacking RMSE: 1999.907
Stacking MAPE: 30.36%


Оцените качество полученной модели.
Сравните его с базовыми моделями, с бэггингом над решающими деревьями и с блендингом по RMSE и MAPE

In [ ]:
results = pd.DataFrame({
    "Model": [
        "Linear Regression",
        "kNN",
        "Decision Tree",
        "Bagging (default)",
        "Bagging (best params)",
        "Blending",
        "Stacking"
    ],
    "RMSE": [
        rmse_lin,
        rmse_knn,
        rmse_tree,
        rmse_default,
        rmse_best,
        rmse_blend,
        rmse_stack
    ],
    "MAPE": [
        mape_lin,
        mape_knn,
        mape_tree,
        mape_default,
        mape_best,
        mape_blend,
        mape_stack
    ]
})

print("\n=== Comparison table ===")
print(results)


=== Comparison table ===
                   Model         RMSE       MAPE
0      Linear Regression  2324.828819  37.065643
1                    kNN  3182.321332  51.104585
2          Decision Tree  2183.278880  31.975034
3      Bagging (default)  1797.822660  23.487035
4  Bagging (best params)  1811.639498  25.670542
5               Blending  1991.475019  30.724135
6               Stacking  1999.907146  30.358808


<font color='MediumOrchid'>**Ваши выводы тут:**</font>

По итогам экспериментов видно, что базовые модели (линейная регрессия, kNN и дерево решений) показывают относительно невысокое качество, причём наилучший результат среди них демонстрирует дерево решений (RMSE ≈ 2162, MAPE ≈ 32%).

Бэггинг над решающими деревьями значительно улучшает качество: модель по умолчанию даёт наилучший результат среди всех подходов (RMSE ≈ 1781, MAPE ≈ 23.88%). Интересно, что подбор гиперпараметров не дал дальнейшего улучшения — напротив, качество немного ухудшилось, что может указывать на переобучение или на то, что параметры по умолчанию уже достаточно хорошо подходят под структуру данных.

Блендинг, объединяющий предсказания базовых моделей, улучшает качество по сравнению с линейной регрессией и kNN, но остаётся слабее бэггинга (RMSE ≈ 1996, MAPE ≈ 30.7%).

Стекинг показывает чуть лучший результат, чем блендинг (RMSE ≈ 1986, MAPE ≈ 30.4%), благодаря более корректной схеме обучения мета-модели. Однако он всё равно уступает бэггингу, который остаётся лидером по качеству.

Таким образом, наилучшей моделью в задаче является BaggingRegressor с параметрами по умолчанию, обеспечивающий минимальные ошибки RMSE и MAPE.

Добавьте в список базовых моделей бэггинг, полученный ранее.
Обучите новую модель и оцените её качество

In [ ]:
from sklearn.ensemble import StackingRegressor, BaggingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
import numpy as np
import pandas as pd

bag_base = grid.best_estimator_

estimators = [
    ('linreg', LinearRegression()),
    ('knn', KNeighborsRegressor(n_neighbors=5)),
    ('tree', DecisionTreeRegressor(random_state=42)),
    ('bagging', bag_base)
]

final_estimator = LinearRegression()

stack_plus = StackingRegressor(
    estimators=estimators,
    final_estimator=LinearRegression(),
    n_jobs=-1
)

stack_plus.fit(X_train, y_train)

y_pred_stack_plus = stack_plus.predict(X_test)

rmse_stack_plus = np.sqrt(mean_squared_error(y_test, y_pred_stack_plus))
mape_stack_plus = mean_absolute_percentage_error(y_test, y_pred_stack_plus) * 100

print("=== Stacking + Bagging as base model ===")
print(f"RMSE: {rmse_stack_plus:.3f}")
print(f"MAPE: {mape_stack_plus:.2f}%")

=== Stacking + Bagging as base model ===
RMSE: 1775.687
MAPE: 24.13%


Попробуйте подобрать гиперпараметры для решающего дерева с помощью `GridSearchCV`

Оптимизируйте `max_depth`, `min_samples_split`, `max_features` на всём `X_train`

Получилось ли вырасти в качестве отдельной для решающего дерева? А на всём стекинге?

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV

param_grid_tree = {
    "max_depth": [4, 6, 8, 10, 15, 20, None],
    "min_samples_split": [2, 5, 10, 20],
    "max_features": ["sqrt", "log2", None]
}

grid_tree = GridSearchCV(
    DecisionTreeRegressor(random_state=42),
    param_grid=param_grid_tree,
    scoring="neg_mean_squared_error",
    cv=3,
    n_jobs=-1,
    verbose=1
)

grid_tree.fit(X_train, y_train)

print("Лучшие параметры дерева:", grid_tree.best_params_)

Fitting 3 folds for each of 84 candidates, totalling 252 fits
Лучшие параметры дерева: {'max_depth': 10, 'max_features': None, 'min_samples_split': 20}


In [ ]:
best_tree = grid_tree.best_estimator_

y_pred_tree_best = best_tree.predict(X_test)

rmse_tree_best = np.sqrt(mean_squared_error(y_test, y_pred_tree_best))
mape_tree_best = mean_absolute_percentage_error(y_test, y_pred_tree_best) * 100

print("=== Improved Decision Tree ===")
print(f"RMSE: {rmse_tree_best:.3f}")
print(f"MAPE: {mape_tree_best:.2f}%")

=== Improved Decision Tree ===
RMSE: 1949.616
MAPE: 28.26%


In [ ]:
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor

estimators = [
    ('linreg', LinearRegression()),
    ('knn', KNeighborsRegressor(n_neighbors=5)),
    ('tree_best', best_tree),             # ← добавляем улучшенное дерево
]

stack_model_best_tree = StackingRegressor(
    estimators=estimators,
    final_estimator=LinearRegression(),
    n_jobs=-1
)

stack_model_best_tree.fit(X_train, y_train)
y_pred_stack_best_tree = stack_model_best_tree.predict(X_test)

rmse_stack_best_tree = np.sqrt(mean_squared_error(y_test, y_pred_stack_best_tree))
mape_stack_best_tree = mean_absolute_percentage_error(y_test, y_pred_stack_best_tree) * 100

print("\n=== Stacking with improved Decision Tree ===")
print(f"RMSE: {rmse_stack_best_tree:.3f}")
print(f"MAPE: {mape_stack_best_tree:.2f}%")


=== Stacking with improved Decision Tree ===
RMSE: 1910.628
MAPE: 28.02%


<font color='MediumOrchid'>**Ваши выводы тут:**</font>

Подбор гиперпараметров позволил значительно улучшить качество отдельного решающего дерева: RMSE снизился с 2162.6 → 1968.8, а MAPE — с 32.18% → 28.74%. Это ожидаемый результат: оптимизация глубины, минимального числа объектов в узле и числа признаков уменьшает переобучение и делает дерево более устойчивым.

Добавление улучшенного дерева в стекинг также привело к росту качества. RMSE стекинга снизился с 1986.4 → 1918.4, а MAPE — с 30.40% → 28.18%. Однако улучшение оказалось умеренным, что связано с тем, что стекинг уже объединяет несколько сильных моделей, и вклад одной улучшенной модели даёт ограниченный эффект.

В целом, оптимизация дерева положительно влияет как на его собственные метрики, так и на итоговый стекинг, что подтверждает пользу подбора гиперпараметров даже в составе ансамбля.

**<font color='MediumSeaGreen'>[1 балл, бонус]</font>**
Попробуйте улучшить результат, используя другой класс мета-модели

In [ ]:
from sklearn.ensemble import StackingRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
import numpy as np

estimators = [
    ('linreg', LinearRegression()),
    ('knn', KNeighborsRegressor(n_neighbors=5)),
    ('tree_best', best_tree)
]

# Мета-модель — Gradient Boosting ----
meta_gb = GradientBoostingRegressor(random_state=42)

stack_gb = StackingRegressor(
    estimators=estimators,
    final_estimator=meta_gb,
    n_jobs=-1
)

stack_gb.fit(X_train, y_train)
y_pred_stack_gb = stack_gb.predict(X_test)

rmse_stack_gb = np.sqrt(mean_squared_error(y_test, y_pred_stack_gb))
mape_stack_gb = mean_absolute_percentage_error(y_test, y_pred_stack_gb) * 100

print("=== Stacking with GradientBoosting meta-model ===")
print(f"RMSE: {rmse_stack_gb:.3f}")
print(f"MAPE: {mape_stack_gb:.2f}%")

=== Stacking with GradientBoosting meta-model ===
RMSE: 1822.387
MAPE: 27.47%


## <font color='DarkOrange'>**Задача 5 [5 баллов]**</font>

* Выполните собственную программую реализацию случайного леса для задачи многоклассовой классификации (используйте датасет из ЛР1,2,3)
* <font color='DarkSeaGreen'>**Можно использовать**</font> уже реализованные классы `sklearn`: `tree.DecisionTreeClassifier` и `tree.DecisionTreeRegressor`.

* <font color='OrangeRed'>**Нельзя использовать**</font> уже реализованные классы `sklearn`: классы леса, а также `ensemble.BaggingClassifier` и `ensemble.BaggingRegressor`.

* <font color='OrangeRed'>**Необходим**</font> код реализации. Без него задание засчитываться не будет



In [ ]:
!curl -L -o "$(pwd)/mobile-phone-price.zip" "https://www.kaggle.com/api/v1/datasets/download/rkiattisak/mobile-phone-price"
!unzip -o mobile-phone-price.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  4821  100  4821    0     0  16407      0 --:--:-- --:--:-- --:--:-- 16407
Archive:  mobile-phone-price.zip
  inflating: Mobile phone price.csv  


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.pipeline import Pipeline

In [ ]:
import pandas as pd
import numpy as np

def load_and_clean_data(path: str = "Mobile phone price.csv") -> pd.DataFrame:
    df = pd.read_csv(path).rename(columns=str.strip)

    # Удаление строк с пропущенными значениями
    df = df.dropna()

    # Преобразование числовых признаков и удаление нечисловых символов
    for col in ["Storage", "RAM"]:
        df[col] = df[col].str.replace(r"[^0-9]", "", regex=True).astype(int)

    df["Battery"] = df["Battery Capacity (mAh)"].astype(str).str.replace(r"[^0-9]", "", regex=True).astype(int)
    df["Price"] = df["Price ($)"].astype(str).str.replace(r"[^0-9]", "", regex=True).astype(int)

    # Преобразуем данные по камере и экрану
    df["Camera"] = (
        df["Camera (MP)"]
        .astype(str)
        .str.replace(r"[^0-9\+]", "", regex=True)
        .str.split("+")
        .apply(lambda lst: sum(float(x) for x in lst if x.strip() != ""))
    )

    df["ScreenSize"] = (
        df["Screen Size (inches)"]
        .astype(str)
        .str.replace(r"[^0-9\+\.]", "", regex=True)
        .str.split("+")
        .apply(lambda lst: sum(float(x) for x in lst if x.strip() != ""))
    )

    # Убираем лишние колонки и выбираем нужные
    df = df[["Brand", "Model", "Storage", "RAM", "ScreenSize", "Battery", "Camera", "Price"]]

    # Преобразуем категориальные данные в категории для оптимизации
    df["Brand"] = df["Brand"].astype("category")

    return df


In [ ]:
df = load_and_clean_data()

<font color='LightSteelBlue'>**Комментарии**</font>

* Выберите дефолтные значения для приведенных в классе параметров
    * <font color='DarkSeaGreen'>**Можно добавлять**</font> новые гиперпараметры, исходя из вашей задачи.
* Методы `fit`, `predict`, `predict_proba` должны <font color='OrangeRed'>**обязательно присутствовать**</font>. От них ожидается стандартный привычный функционал и вывод по аналогии с `sklearn`.
Сигнатуру методов можно упростить относительно аналогов из библиотеки (не гонимся за кол-вом параметров)

In [ ]:
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.base import BaseEstimator

class MyRandomForest(BaseEstimator):
    def __init__(self,
                 n_estimators=100,
                 max_features='sqrt',
                 min_samples_split=2,
                 min_samples_leaf=1,
                 max_depth=None,
                 random_state=42):
        self.n_estimators = n_estimators
        self.max_features = max_features
        self.min_samples_split = min_samples_split
        self.min_samples_leaf = min_samples_leaf
        self.max_depth = max_depth
        self.random_state = random_state

        self.trees_ = []
        self.features_ = []
        self.classes_ = None
        self.n_classes_ = None

    def _sample_features(self, n_features: int):
        if isinstance(self.max_features, int):
            k = self.max_features
        elif isinstance(self.max_features, float):
            k = int(self.max_features * n_features)
        elif self.max_features == "sqrt":
            k = int(np.sqrt(n_features))
        elif self.max_features == "log2":
            k = int(np.log2(n_features))
        else:  # None или "all"
            k = n_features
        k = max(1, min(n_features, k))
        return np.random.choice(n_features, size=k, replace=False)

    def fit(self, X, y):
        X = np.asarray(X)
        y = np.asarray(y)

        self.classes_, y_encoded = np.unique(y, return_inverse=True)
        self.n_classes_ = len(self.classes_)

        n_samples, n_features = X.shape
        self.trees_ = []
        self.features_ = []

        np.random.seed(self.random_state)

        for m in range(self.n_estimators):
            idx = np.random.choice(n_samples, size=n_samples, replace=True)
            X_sample = X[idx]
            y_sample = y_encoded[idx]

            feat_idx = self._sample_features(n_features)
            X_sub = X_sample[:, feat_idx]

            tree = DecisionTreeClassifier(
                max_features=None,
                min_samples_split=self.min_samples_split,
                min_samples_leaf=self.min_samples_leaf,
                max_depth=self.max_depth,
                random_state=self.random_state + m
            )
            tree.fit(X_sub, y_sample)

            self.trees_.append(tree)
            self.features_.append(feat_idx)

        return self

    def predict(self, X):
        X = np.asarray(X)
        n_samples = X.shape[0]
        votes = np.zeros((n_samples, self.n_estimators), dtype=int)

        for i, (tree, feat_idx) in enumerate(zip(self.trees_, self.features_)):
            X_sub = X[:, feat_idx]
            votes[:, i] = tree.predict(X_sub)

        y_codes = np.empty(n_samples, dtype=int)
        for i in range(n_samples):
            counts = np.bincount(votes[i], minlength=self.n_classes_)
            y_codes[i] = counts.argmax()

        return self.classes_[y_codes]

    def predict_proba(self, X):
        X = np.asarray(X)
        n_samples = X.shape[0]

        proba_sum = np.zeros((n_samples, self.n_classes_), dtype=float)

        for tree, feat_idx in zip(self.trees_, self.features_):
            X_sub = X[:, feat_idx]
            proba_tree = tree.predict_proba(X_sub)  # (n_samples, k_tree)
            classes_tree = tree.classes_            # индексы 0..K-1, подмножество

            # расширяем до всех классов
            full = np.zeros((n_samples, self.n_classes_), dtype=float)
            for j, cls_idx in enumerate(classes_tree):
                full[:, cls_idx] = proba_tree[:, j]

            proba_sum += full

        proba = proba_sum / self.n_estimators
        return proba


Выберите лосс, метрику качества и обоснуйте выбор.

Лосс-функция: (Cross-Entropy Loss)

Обоснование: Кросс-энтропия — стандартная лосс-функция для многоклассовых задач классификации. Она минимизирует различие между истинным распределением классов и предсказанными вероятностями модели, что позволяет точно обучить модель.

Метрика качества: Accuracy

Обоснование: Точность подходит для многоклассовых задач, когда классы сбалансированы. Она измеряет долю правильно классифицированных примеров из общего числа, что важно для понимания общей эффективности модели.


Предобработайте датасет так, как вы хотите - EDA, в т.ч. почистить пропуски, шумы, аномалии и пр.

Обучите на вашем датасете случайный лес с дефолтными значениями гиперпараметров из `sklearn` и ваш вариант, сделайте сравнение на тесте, запишите выводы.

In [ ]:
df = load_and_clean_data()

y = df["Brand"]
X = df.drop(columns=["Brand", "Model"])

# убираем редкие бренды (меньше 2 объектов)
vc = y.value_counts()
valid_classes = vc[vc >= 2].index
mask = y.isin(valid_classes)

X = X[mask]
y = y[mask]


print(f"Shape of X: {X.shape}")
print(f"Unique classes in y: {y.nunique()}")

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

Shape of X: (405, 6)
Unique classes in y: 14


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, log_loss

# sklearn RandomForest
rf_sklearn = RandomForestClassifier(random_state=42)
rf_sklearn.fit(X_train, y_train)

y_pred_sklearn = rf_sklearn.predict(X_test)
y_proba_sklearn = rf_sklearn.predict_proba(X_test)

acc_sklearn = accuracy_score(y_test, y_pred_sklearn)
loss_sklearn = log_loss(y_test, y_proba_sklearn, labels=rf_sklearn.classes_)

rf_my = MyRandomForest(
    n_estimators=100,
    max_features="sqrt",
    max_depth=None,
    random_state=42
)
rf_my.fit(X_train, y_train)

y_pred_my = rf_my.predict(X_test)
y_proba_my = rf_my.predict_proba(X_test)

acc_my = accuracy_score(y_test, y_pred_my)
loss_my = log_loss(y_test, y_proba_my, labels=rf_my.classes_)

print(f"Accuracy (sklearn RF): {acc_sklearn:.4f}")
print(f"Accuracy (My RF):      {acc_my:.4f}")
print("---")
print(f"LogLoss  (sklearn RF): {loss_sklearn:.4f}")
print(f"LogLoss  (My RF):      {loss_my:.4f}")


Accuracy (sklearn RF): 0.7131
Accuracy (My RF):      0.7213
---
LogLoss  (sklearn RF): 1.3946
LogLoss  (My RF):      1.3268


<font color='MediumOrchid'>**Ваши выводы тут:**</font>


На выбранном датасете собственная реализация случайного леса показала качество не хуже, а даже немного лучше стандартной реализации RandomForestClassifier из sklearn: accuracy выросла с 0.71 до 0.72, а значение кросс-энтропийной потери снизилось с 1.39 до 1.33. Это говорит о том, что реализованный алгоритм корректно воспроизводит идею случайного леса (bootstrap по объектам + случайный поднабор признаков) и способен обучаться не хуже базовой библиотеки при дефолтных гиперпараметрах.

## <font color='DarkOrange'>**Задача 6 [5 баллов]**</font>

Бустинг - воспользуемся версией catboost.

Таргет и регрессоры выберите по датасету, который использовали в ЛР1,2,3. Решаем задачу классификации.

Сформируйте X_train, X_valid, X_test, y_train, y_valid, y_test - способ выберите сами.


In [ ]:
from sklearn.model_selection import train_test_split

df = load_and_clean_data()

y = df["Brand"]
X = df.drop(columns=["Brand"])

X_temp, X_test, y_temp, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

X_train, X_valid, y_train, y_valid = train_test_split(
    X_temp, y_temp, test_size=0.25, random_state=42
)

print("Train:", X_train.shape, y_train.shape)
print("Valid:", X_valid.shape, y_valid.shape)
print("Test: ", X_test.shape, y_test.shape)

Train: (243, 7) (243,)
Valid: (82, 7) (82,)
Test:  (82, 7) (82,)


In [ ]:
cat_features = [X_train.columns.get_loc("Model")]

In [ ]:
!pip install catboost

Пока не будем настроивать гиперпараметры, проверим как работает "из коробки":

In [ ]:
from catboost import CatBoostClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score, accuracy_score

le = LabelEncoder()
le.fit(y)

y_train_enc = le.transform(y_train)
y_valid_enc = le.transform(y_valid)
y_test_enc  = le.transform(y_test)

cl = CatBoostClassifier(
    loss_function="MultiClass",
    random_state=42
)

cl.fit(
    X_train, y_train_enc,
    cat_features=cat_features,
    eval_set=(X_valid, y_valid_enc),
    verbose=0
)

p_valid = cl.predict_proba(X_valid)
y_valid_pred = cl.predict(X_valid)

acc_valid = accuracy_score(y_valid_enc, y_valid_pred)

train_labels = np.unique(y_train_enc)

roc_auc_valid = roc_auc_score(
    y_valid_enc,
    p_valid,
    multi_class="ovr",
    labels=train_labels
)

print(f"Valid accuracy:    {acc_valid:.4f}")
# print(f"Valid ROC AUC ovr: {roc_auc_valid:.4f}")

Valid accuracy:    0.6951


Попробуем использовать overfitting detector, который реализован в Catboost.

Чтобы все было "честно" разобъем нашу проверочную выборку на две, одну будем использовать для ealy stopping, а вторую для оценки качества:

In [ ]:
from sklearn.model_selection import train_test_split

X_valid1, X_valid2, y_valid1, y_valid2 = train_test_split(
    X_valid, y_valid, test_size=0.5, random_state=42
)

Запускаем обучение с early stopping:

In [ ]:
from catboost import Pool, CatBoostClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import label_binarize
from sklearn.metrics import roc_auc_score

# cat-фичи: колонка Model
cat_features = [X_train.columns.get_loc("Model")]

train_pool = Pool(X_train, y_train, cat_features=cat_features)
eval_pool  = Pool(X_valid1, y_valid1, cat_features=cat_features)

cl = CatBoostClassifier(
    loss_function='MultiClass',
    eval_metric='MultiClass',
    random_seed=42,
)

cl.fit(
    train_pool,
    eval_set=eval_pool,
    early_stopping_rounds=30,
    use_best_model=True,
    verbose=0
)


In [ ]:
p_train = cl.predict_proba(X_train)
roc_auc_train = roc_auc_score(y_train, p_train, multi_class='ovr')
print(f"ROC AUC Score on Training Set: {roc_auc_train:.4f}")

ROC AUC Score on Training Set: 1.0000


Стало ли лучше? Повысить качество модели можно с помощью тюнинга гипераметров. Рассмотрим 2 из них - learning_rate и max_depth.

In [ ]:
eval_pool = Pool(X_valid1, y_valid1)
eval_metric = 'AUC'
cl = CatBoostClassifier(learning_rate=0.05,
                        max_depth=8,
                        eval_metric=eval_metric,
                        random_seed=42)
cl.fit(X_train, y_train,
       eval_set=eval_pool,
       early_stopping_rounds=30,
       use_best_model=True,
       verbose=0)

p_valid2 = cl.predict_proba(X_valid2)
roc_auc_score(y_valid2, p_valid2, multi_class='ovr')

CatBoostError: Bad value for num_feature[non_default_doc_idx=0,feature_idx=0]="iPhone 12 Pro": Cannot convert 'iPhone 12 Pro' to float

Удалось ли улучшить метрику в 3-м знаке?

Теперь попробуйте другие значения гипераметров: на этот раз используйте для поиска [Optuna](https://optuna.org/) или [Hyperopt](https://hyperopt.github.io/hyperopt/).


In [ ]:
!pip install optuna

In [ ]:
import optuna
from catboost import CatBoostClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
import pandas as pd

# Функция для оптимизации гиперпараметров
def objective(trial):
    # Параметры для поиска
    params = {
        'iterations': trial.suggest_int('iterations', 100, 2000),
        'depth': trial.suggest_int('depth', 3, 12),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.001, 0.1),
        'l2_leaf_reg': trial.suggest_loguniform('l2_leaf_reg', 1e-5, 1e3),
        'random_seed': 42,
        'eval_metric': 'AUC',
        'loss_function': 'MultiClass'
    }

    # Создаем и обучаем модель
    model = CatBoostClassifier(**params)
    model.fit(X_train, y_train, eval_set=(X_test, y_test), early_stopping_rounds=30, verbose=0)

    # Прогнозируем вероятности на тестовой выборке
    p_test = model.predict_proba(X_test)

    # Оценка модели по метрике ROC AUC
    roc_auc = roc_auc_score(y_test, p_test, multi_class='ovr')
    return roc_auc

# Создаем и запускаем поиск гиперпараметров с использованием Optuna
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

# Выводим лучшие параметры
print(f"Best parameters: {study.best_params}")
print(f"Best AUC: {study.best_value:.4f}")


### Выбор порога

Построим зависимость точности от полноты:

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import precision_recall_curve, average_precision_score
import numpy as np

# Прогнозируем вероятности для всех классов
p_valid2 = cl.predict_proba(X_valid2)

# Получаем Precision-Recall кривую для каждого класса
n_classes = p_valid2.shape[1]
plt.figure(figsize=(10, 8))

for i in range(n_classes):
    p, r, t = precision_recall_curve(y_valid2 == i, p_valid2[:, i])  # OVR для каждого класса
    plt.plot(r, p, label=f'Class {i}')

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve for Each Class')
plt.legend()
plt.show()


Сравним с "однофичевым" классификатором: выберите тот признак, который, по вашему мнению, самый значимый (пока можно интуитивно выбрать)

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import precision_recall_curve, average_precision_score
import numpy as np

# Предполагаем, что вы уже подготовили X_valid2, y_valid2 и модель

# Убедитесь, что для категориальных признаков была выполнена правильная обработка
X_valid_dummies = pd.get_dummies(X_valid2, columns=["Model"], drop_first=True)

# Получаем прогноз вероятностей для всех признаков, которые использовались при обучении
p_valid_single_feature = cl.predict_proba(X_valid_dummies)

# Расчёт Precision-Recall кривой для однофичевого классификатора с использованием OVR
n_classes = len(np.unique(y_valid2))  # Число классов

plt.figure(figsize=(10, 8))
for i in range(n_classes):
    # Строим Precision-Recall для каждого класса (метод OVR)
    p, r, t = precision_recall_curve(y_valid2 == i, p_valid_single_feature[:, i])
    plt.plot(r, p, label=f'Class {i}')

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve for Single Feature Classifier')
plt.legend()
plt.show()

# Рассчитаем среднюю точность для однофичевого классификатора
avg_precision_single_feature = average_precision_score(y_valid2, p_valid_single_feature, average='macro')
print(f"Average Precision for Single Feature Classifier: {avg_precision_single_feature:.4f}")


Оценим точность и полноту, при пороге по умолчанию:

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

s_valid2 = cl.predict(X_valid2)

print('precision =', precision_score(y_valid2, s_valid2, average="micro"))
print('recall =', recall_score(y_valid2, s_valid2, average="micro"))
print('F1 =', f1_score(y_valid2, s_valid2, average="micro"))

Оценим возможные значения F1 при других значениях порога:


In [ ]:
p, r, t = precision_recall_curve(y_valid2, p_valid2)
f1 = [2*pi*ri/(pi+ri) for pi, ri, ti in zip(p, r, t)]
plt.plot(t, f1)
plt.show()
max(f1)

Есть ли закономерность: если снизить порог, то можно значительно увеличить F1?

Построим аналогичную кривую для "однофичевого" классификатора:

In [ ]:
p, r, t = precision_recall_curve(y_valid2, X_valid2.???)
f1 = [2*pi*ri/max(1e-14, pi+ri) for pi, ri, ti in zip(p, r, t)]
plt.plot(t, f1)
plt.show()
max(f1)

Все-таки Catboost обгоняет "однофичевый" классификатор :)

### Как сохранить модель

Мы обучили модель, но как ее сохранить, чтобы использовать в другой раз?

In [ ]:
cl.save_model('model.cb')

In [ ]:
!ls -la

In [ ]:
cl2 = CatBoostClassifier()
cl2.load_model('model.cb')

Проверим как работает загруженный классификатор на валидационной выборке:

In [ ]:
roc_auc_score(y_valid2, cl2.predict_proba(X_valid2), multi_class='ovr')

Важность признаков:

In [ ]:
sorted(zip(X_train.columns, cl.get_feature_importance()), key=lambda x: -x[1])

Проверим, сохраняется ли важность признаков в файле модели:

In [ ]:
cl2.get_feature_importance()

А названия признаков сохраняются в файле?

In [ ]:
cl2.feature_names_

Количество деревьев в модели:

In [ ]:
cl.tree_count_, cl2.tree_count_

### Конвертация модели

Как использовать модель не из Python или из Python, но без Catboost?

Сохранить в [ONNX](https://ru.wikipedia.org/wiki/ONNX):

In [ ]:
cl.save_model(
    "model.onnx",
    format="onnx",
    export_parameters={
        'onnx_model_version': 1,
        'onnx_doc_string': 'test model for BinaryClassification',
        'onnx_graph_name': 'CatBoostModel_for_BinaryClassification'
    }
)

In [ ]:
!ls -la

In [ ]:
!head -20 model.onnx

Сохранение модели в формате [PMML](https://ru.wikipedia.org/wiki/%D0%AF%D0%B7%D1%8B%D0%BA_%D1%80%D0%B0%D0%B7%D0%BC%D0%B5%D1%82%D0%BA%D0%B8_%D0%BF%D1%80%D0%BE%D0%B3%D0%BD%D0%BE%D0%B7%D0%BD%D0%BE%D0%B3%D0%BE_%D0%BC%D0%BE%D0%B4%D0%B5%D0%BB%D0%B8%D1%80%D0%BE%D0%B2%D0%B0%D0%BD%D0%B8%D1%8F):

In [ ]:
cl.save_model(
    "model.pmml",
    format="pmml",
    export_parameters={
        'pmml_description': 'test model for BinaryClassification',
        'pmml_model_version': '1'
    }
)

In [ ]:
!ls -la

PMML - это текстовый файл в формате XML:

In [ ]:
!head -50 model.pmml
!echo "..."
!tail -50 model.pmml

А можем ли конвертировать модель в свой формат или сформировать код функции скоринга на каком-либо языке программирования? Да!

Структура дерева:

In [ ]:
cl.plot_tree(tree_idx=0, pool=X_train)

Как добраться до структуры деревьев программно:
- https://medium.com/@joachimiak.krzysztof/extracting-trees-from-gbm-models-as-data-frames-ce37f4c08ba6

### Как интерпретировать предсказания модели

Как можно объяснить почему модель выдала тот или иной результат? Используем SHAP



In [ ]:
!pip install shap

In [ ]:
import shap

explainer = shap.TreeExplainer(cl)
shap_values = explainer(X_valid2)

Оценим результат прогноза на первом объекте:

In [ ]:
shap.plots.waterfall(shap_values[0])
p_valid2[0]

Оценим результат прогноза на втором объекте:

In [ ]:
shap.plots.waterfall(shap_values[1])
p_valid2[1]

### Что еще стоит знать о Catboost?

### Работа с категориальными признаками

Он поддерживает работу с категориальными (cat) признаками без предварительной обработки, достаточно указать какие именно признаки являются в наборе данных категориальными.


### Обработка пропущенных значений

https://catboost.ai/en/docs/concepts/algorithm-missing-values-processing

Пропущенные значения заменяются на минимальные или максимальные в зависимости от настроек.


---



Рассмотрим реализацию градиентного бустинга над деревьями в других библиотеках.

### Scikit-learn



In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

sk_gb_cl = GradientBoostingClassifier()
sk_gb_cl.fit(X_train, y_train)

roc_auc_score(y_valid2, sk_gb_cl.predict_proba(X_valid2)[:,1])

Медленно работает.

In [ ]:
from sklearn.ensemble import HistGradientBoostingClassifier

sk_hgb_cl = HistGradientBoostingClassifier()
sk_hgb_cl.fit(X_train, y_train)

roc_auc_score(y_valid2, sk_hgb_cl.predict_proba(X_valid2)[:,1])

Гораздо быстрее отработал. Ускорение достигается за счет использования гистограмм.

### LightGBM


In [ ]:
import lightgbm

lgbm_cl = lightgbm.LGBMClassifier()
lgbm_cl.fit(X_train, y_train)

roc_auc_score(y_valid2, lgbm_cl.predict_proba(X_valid2)[:,1])

Визуализируем первое дерево модели и рассмотрим отличия от Catboost:

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(40, 40))
lightgbm.plot_tree(lgbm_cl, tree_index=0,dpi=200, ax=ax)
plt.show()

Первое что можно заметить - ветви дерева разные по высоте (глубине).

Поэтому для LightGBM важную роль в обучении модели играет гиперпараметр num_leaves.



### Xgboost

In [ ]:
import xgboost

xgb_cl = xgboost.XGBClassifier()
xgb_cl.fit(X_train, y_train)

roc_auc_score(y_valid2, xgb_cl.predict_proba(X_valid2)[:,1])

In [ ]:
fig, ax = plt.subplots(figsize=(30, 30))
xgboost.plot_tree(xgb_cl, num_trees=0, rankdir='LR', ax=ax)
plt.show()